# Stochastic Depth (Residual Block Drop) — Temelden Derine (PyTorch)

Bu notebook şunları bitirir ✅  
1) **Stochastic Depth nedir?** (mantık + amaç)  
2) **DropPath ile ilişkisi** (aynı aile, farklı genelleme)  
3) Matematiksel formül (inverted scaling)  
4) **Sıfırdan implementasyon** (`stochastic_depth` + `StochasticDepth` module)  
5) **Residual bloğa entegrasyon** (doğru yer: branch)  
6) **Derinlik boyunca drop rate schedule** (linear decay)  
7) Mini model örneği + sanity check

> Kısa tanım: Stochastic Depth, eğitim sırasında bazı residual blokları **rastgele bypass** eder.


## 0) Problem: Derin ağlarda neden işe yarar?

Derin ağlarda (ResNet gibi) şu risk var:
- Model, çok derin olunca **overfit** ve **optimizasyon zorluğu** artar.
- Her blok sürekli aktif olduğu için model belirli blok kombinasyonlarına aşırı bağımlı kalabilir.

**Stochastic Depth** fikri:
- Eğitimde bazı blokları kapat (skip et)
- Model, farklı derinliklerde “alt ağlar” ile öğrenmiş gibi olur (**implicit ensemble**)
- Testte tüm bloklar aktif kalır (tam kapasite)

Bu Dropout’un "neuron-level" karşılığı değil; **block-level** bir regularization.


## 1) Stochastic Depth vs DropPath (çok net)

- **Stochastic Depth**: genelde **residual bloğun tamamını** (branch'i) düşürür.
- **DropPath**: daha genel isim; herhangi bir **path/branch** düşürebilir (Transformer/MBConv vs).

ResNet için çoğu pratikte:
> Stochastic Depth ≈ DropPath (residual branch üzerinde)

Senin DropPath implementasyonun varsa, Stochastic Depth onun **özel hali** gibi düşünebilirsin.


## 2) Matematik (inverted scaling)

Residual yapı:
\[
y = x + F(x)
\]

Stochastic Depth'te eğitim sırasında:
- Olasılık \(q = 1-p\) ile branch tutulur
- Olasılık \(p\) ile branch kapatılır (F(x)=0)

Mask:
\[
m \sim Bernoulli(q) \quad (m \in \{0,1\})
\]

İnverted scaling (beklenen değeri korumak için):
\[
y = x + \frac{m}{q} F(x)
\]

- Eğer \(m=0\) → \(y=x\)  (blok bypass)
- Eğer \(m=1\) → \(y=x + F(x)/q\)

Eval modunda:
\[
y = x + F(x)
\]


In [ ]:

# ===== 3) Imports & seed =====
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


## 3) Sıfırdan: stochastic_depth fonksiyonu

Bu fonksiyon **residual branch çıktısı** üzerinde çalışır.

Girdi:
- `x` = branch çıktısı (F(x))  -> shape: [B, C, H, W] (veya farklı)
- `p` = drop probability
- `training` = sadece train'de aktif

Çıktı:
- Ya 0'lanmış branch
- Ya da ölçeklenmiş branch ( / q )


In [ ]:

def stochastic_depth(x: torch.Tensor, p: float, training: bool) -> torch.Tensor:
    """Stochastic Depth / DropPath on a residual branch output.
    x: residual branch output (F(x))
    p: drop probability
    """
    if not (0.0 <= p < 1.0):
        raise ValueError("p must be in [0, 1).")
    if (not training) or p == 0.0:
        return x

    q = 1.0 - p  # keep prob

    # Mask'i batch bazında üretiriz: [B, 1, 1, 1] -> tüm kanallara ve alana broadcast
    # (ResNet Stochastic Depth'in tipik hali: sample-wise drop)
    shape = (x.size(0),) + (1,) * (x.ndim - 1)
    mask = torch.empty(shape, device=x.device, dtype=x.dtype).bernoulli_(q)

    # inverted scaling: beklenen değer korunur
    return x * mask / q

# sanity check
B, C, H, W = 4, 8, 16, 16
branch = torch.randn(B, C, H, W)
out = stochastic_depth(branch, p=0.5, training=True)
out.shape


### 3.1) Maskeyi gözlemek (hangi örneklerde branch düştü?)

Maske sample-wise olduğu için, bazı batch örnekleri tamamen 0'lanır.


In [ ]:

# Hangi örnekler düştü? (tam 0 olanları yakalayalım)
p = 0.5
branch = torch.randn(6, 4, 8, 8)
d = stochastic_depth(branch, p=p, training=True)

energy = d.abs().mean(dim=(1,2,3))  # her örnek için enerji
energy


## 4) nn.Module olarak: StochasticDepth

`model.train()` iken çalışır, `model.eval()` iken identity davranır (branch'i dokunmadan bırakır).


In [ ]:

class StochasticDepth(nn.Module):
    def __init__(self, p: float = 0.1):
        super().__init__()
        if not (0.0 <= p < 1.0):
            raise ValueError("p must be in [0, 1).")
        self.p = float(p)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return stochastic_depth(x, p=self.p, training=self.training)

sd = StochasticDepth(p=0.5)
sd.train()
a = sd(torch.randn(4,3,8,8))
sd.eval()
b = sd(torch.randn(4,3,8,8))
a.shape, b.shape


## 5) Entegrasyon: Residual bloğa doğru yerleştirme

**Kural:** Stochastic Depth, **residual branch (F(x))** üzerinde uygulanır.

Yanlış:
- skip yoluna uygulamak (identity'yi bozarsın)
- residual toplama öncesi değil de, toplama sonrası rastgele sıfırlamak (felsefe değişir)

Doğru akış:
1) `out = F(x)`
2) `out = SD(out)`
3) `y = x + out`


In [ ]:

class BasicResBlockSD(nn.Module):
    def __init__(self, cin: int, cout: int, stride: int = 1, sd_p: float = 0.1, act: str = "relu"):
        super().__init__()
        self.conv1 = nn.Conv2d(cin, cout, 3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(cout)

        self.conv2 = nn.Conv2d(cout, cout, 3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(cout)

        self.act = nn.ReLU(inplace=True) if act == "relu" else nn.SiLU(inplace=True)

        # Stochastic Depth module (branch üzerinde)
        self.sd = StochasticDepth(p=sd_p)

        # shortcut/proj
        self.shortcut = nn.Identity()
        if stride != 1 or cin != cout:
            self.shortcut = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout)
            )

    def forward(self, x):
        identity = self.shortcut(x)

        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        # SD -> branch drop / scale
        out = self.sd(out)

        out = out + identity
        out = self.act(out)
        return out

blk = BasicResBlockSD(16, 16, sd_p=0.5).to(device)
blk.train()
y = blk(torch.randn(4,16,32,32, device=device))
y.shape


## 6) Drop rate schedule (derinlik boyunca lineer artış)

Pratikte Stochastic Depth şöyle uygulanır:
- Erken bloklar: düşük drop (p küçük)
- Derin bloklar: yüksek drop (p büyük)

Çünkü:
- Erken bloklar temel feature çıkarır; sık düşürmek istemezsin
- Derin bloklar daha spesifik; regularize etmek daha mantıklı

Lineer schedule örneği:
\[
p_i = p_{max} \cdot \frac{i}{L-1}
\]
(i: blok index, L: toplam blok sayısı)


In [ ]:

def make_sd_probs(num_blocks: int, p_max: float):
    if num_blocks <= 1:
        return [p_max]
    return [p_max * i / (num_blocks - 1) for i in range(num_blocks)]

make_sd_probs(6, 0.2)


## 7) Mini model: Stage'lerde SD schedule ile kullan

Bu model:
- stem
- 6 residual block
- her block için farklı sd_p (lineer)
- GAP + FC

Amaç: entegrasyon kalıbını görmek.


In [ ]:

class TinyResNetWithStochasticDepth(nn.Module):
    def __init__(self, num_classes=10, base_channels=32, blocks=6, sd_max=0.2):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, base_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(base_channels),
            nn.ReLU(inplace=True)
        )

        probs = make_sd_probs(blocks, sd_max)
        layers = []
        c = base_channels
        for i in range(blocks):
            # örnek amaçlı: 3 bloktan sonra downsample
            if i == blocks // 2:
                layers.append(BasicResBlockSD(c, c*2, stride=2, sd_p=probs[i]))
                c *= 2
            else:
                layers.append(BasicResBlockSD(c, c, stride=1, sd_p=probs[i]))
        self.blocks = nn.Sequential(*layers)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(c, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)

model = TinyResNetWithStochasticDepth(num_classes=100, blocks=6, sd_max=0.3).to(device)
model.train()
logits = model(torch.randn(2,3,32,32, device=device))
logits.shape


## 8) Pratik ayarlar (senin repo için net öneri)

- Başlangıç `sd_max`: **0.05 – 0.2**
- Çok derin modelde (50+ layer) `sd_max`: **0.2 – 0.5** (dataset/augment/WD’e bağlı)
- Dropout/SpatialDropout ile birlikte kullanacaksan:
  - SD (block-level) ana regularizer olur
  - activation-level dropout’u düşük tut (p≈0.05–0.15)

**Yerleşim:**  
- `F(x)` çıktısına, `x + F(x)` toplamadan hemen önce.

Sıradaki adım (istersen):  
- Bu SD’yi senin mevcut `DropPath` kodunla aynı API’ye bağlamak (tek modül, iki mod).
